Description: Detect edge & evaluate FOM of proposed method with various noise level

Python version: 3.9

In [23]:
# Link Google colab notebook to your personal google drive account.

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# Install packages
!pip install ruptures
!pip install skan
!pip install aicsimageio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
# %% Import libraries
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io  # v.0.18.1
from skimage import filters
from skimage.morphology import disk
from skimage import io
from pathlib import Path
from os import sep
from cv2 import morphologyEx, MORPH_OPEN
import sys
from aicsimageio.writers.ome_tiff_writer import OmeTiffWriter
from aicsimageio import AICSImage

In [44]:
# Import original module
project_path = f"{sep}content{sep}drive{sep}My Drive{sep}my_project{sep}"
sys.path.append(project_path)

from cpseg_module.cpseg_functions_colab_v2 import ak_rotate_edge_overlay, postprocess, connect_ends

In [45]:
# Input image file names in /content/drive/my_project
imFile = "test_data_4d.tif"

In [46]:
# %% Parameters
# Image file
rootfile = f"{sep}content{sep}drive{sep}My Drive{sep}my_project{sep}"
# Get image file name
imFilename = f"{rootfile}{imFile}"              # Image file path                                    
se_size = 200                                   # Background subtraction
medianValue = [5, 5]#[30, 30]                   # Median filter
y_interval = 2  # 5                             # Sparse scan along y-axis, pixel
rotation_angle = 4 #3                           # 0: 45 degree, 3: semicircle
                                                # 1: 30 degree, 4: semicircle
                                                # 2: 15 degree, 5: semicircle
changepoint_algo = 1                            # 0: number known, 1: number not known
numBorder = 4                                   # Number of change points,
                                                # if changepoint_algo = 1, use PELT for panelized detection
penalty_value = 100000                          # Penalty cost when numBorder = 0
numlayer = 1                                    # 0: single <- avoid in this workflow
                                                # 1: overlay multiple
model = "linear"                                # "rbf", "linear", "cosine"
smallNoiseRemov = 1                             # 0:No noise removal
                                                # 1:Connect border, then remove small noises
                                                # 2:Remove small noisess first, then connect
senoise = y_interval * 2                        # Structural element size of morphology opening:default:5
neib = 25                                       # Remove small objects less than neib(pixel) default:100
select_biggest = 0                              # 0: No selection; 1: select biggest
nskeletonize = 1                                # 0: No skeletonization,
                                                # 1: with skeletonization
postProcessing =    1                           # 0: No
                                                # 1: Yes
min_j2e_size = 100                              # Remove branch smaller than min_j2e_size                                                  #(pixels), default 1000000 pix
connect_gap = 1                                 # 0: No
                                                # 1: Yes
implot =            0                           # 0: No image
                                                # 1: Plot image
save_data =         1                           # 0: Do not save data
                                                # 1: Save data

In [54]:
# %% Processing
 # Load image
I0 = AICSImage(imFilename)
I0 = I0.data

# Convert images to 3d
I0_1 = np.copy(I0)
num_image = np.shape(I0)[0] * np.shape(I0)[2]
I0_1 = np.reshape(I0, (num_image, np.shape(I0)[-2], np.shape(I0)[-1]))
BW_proposed_pp = np.zeros_like(I0_1) # Edge containers

for iz in np.arange(num_image):
    print(F'Image number is {iz}.')
    Iori = I0_1[iz,:,:]

    # Pre-processing
    se_imopen = disk(se_size)                                           # structrul element for morph open
    background = morphologyEx(Iori, MORPH_OPEN, se_imopen)      # opening
    I_bg = Iori - background                                            # Subtract background
    Ien = filters.median(I_bg, disk(medianValue[0]))                  # Smoothing filter

    # Edge detection
    def proposed():
        BW = ak_rotate_edge_overlay\
            (Ien,numlayer,rotation_angle,numBorder,model,y_interval,penalty_value, changepoint_algo)
        print('Edge detected.')

        # Post-processing
        if postProcessing == 0:
            BWlast = np.copy(BW)
        elif postProcessing == 1:
            BWlast = postprocess(BW, min_j2e_size)
            if connect_gap == 1:
                BWlast = connect_ends(BWlast, min_j2e_size)

        BW_proposed_pp[iz, :, :] = BWlast

        # Plot images
        if implot == 1:
            fig, ax = plt.subplots(nrows=3, ncols=1, tight_layout=False, figsize=(10, 10), sharex=True, sharey=True)
            ax[0].imshow(Iori, cmap='gray')
            ax[1].imshow(BW, cmap='gray_r')
            ax[2].imshow(BWlast, cmap='gray_r')

            for irows in np.arange(3):
                ax[irows].set_xticks([])
                ax[irows].set_yticks([])

            plt.show()
    proposed()

# Reversely convert to 4D
BW_proposed_pp = np.reshape(BW_proposed_pp, (np.shape(I0)[0], np.shape(I0)[2], np.shape(I0)[-2], np.shape(I0)[-1]),order='C')

(3, 1, 3, 668, 695)
<class 'numpy.ndarray'>
(3, 1, 3, 668, 695)
Image number is 0.
Edge detected.
prune repeat: 1
prune repeat: 2
Image number is 1.
Edge detected.
prune repeat: 1
prune repeat: 2
prune repeat: 3
Image number is 2.
Edge detected.
prune repeat: 1
prune repeat: 2
Image number is 3.
Edge detected.
prune repeat: 1
prune repeat: 2
Image number is 4.
Edge detected.
prune repeat: 1
prune repeat: 2
prune repeat: 3
Image number is 5.
Edge detected.
prune repeat: 1
prune repeat: 2
prune repeat: 3
Image number is 6.
Edge detected.
prune repeat: 1
prune repeat: 2
prune repeat: 3
Image number is 7.
Edge detected.
prune repeat: 1
prune repeat: 2
prune repeat: 3
Image number is 8.
Edge detected.
prune repeat: 1


In [60]:
# Save edge
result_path = f"{project_path}result{sep}"
save_file = f"{result_path}edge.tif"
OmeTiffWriter.save(BW_proposed_pp, save_file, dim_order="TZYX")

/usr/local/lib/python3.8/dist-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
